# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

In [ ]:
#Your code here; create the connection and cursor

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [4]:
c.execute('''select customerNumber, contactLastName, contactFirstName from customers 
            where customerNumber in (select customerNumber from orders where orderDate= '2003-01-31');''').fetchall()

[('141', 'Freyre', 'Diego ')]

## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [25]:
c.execute('''select productName, count(orderNumber) as numorders, sum(quantityOrdered) as uogs 
            from products 
            join orderdetails using(productCode) group by productName order by uogs desc;''').fetchall()

[('1992 Ferrari 360 Spider red', 53, 1808),
 ('1937 Lincoln Berline', 28, 1111),
 ('American Airlines: MD-11S', 28, 1085),
 ('1941 Chevrolet Special Deluxe Cabriolet', 28, 1076),
 ('1930 Buick Marquette Phaeton', 28, 1074),
 ('1940s Ford truck', 28, 1061),
 ('1969 Harley Davidson Ultimate Chopper', 28, 1057),
 ('1957 Chevy Pickup', 28, 1056),
 ('1964 Mercedes Tour Bus', 28, 1053),
 ('1956 Porsche 356A Coupe', 27, 1052),
 ('Corsair F4U ( Bird Cage)', 28, 1051),
 ('F/A 18 Hornet 1/72', 28, 1047),
 ('1980s Black Hawk Helicopter', 28, 1040),
 ('1913 Ford Model T Speedster', 28, 1038),
 ('1997 BMW R 1100 S', 28, 1033),
 ('1972 Alfa Romeo GTA', 28, 1030),
 ('1962 Volkswagen Microbus', 28, 1029),
 ('2002 Suzuki XREO', 28, 1028),
 ('The USS Constitution Ship', 27, 1020),
 ('2001 Ferrari Enzo', 27, 1019),
 ('1960 BSA Gold Star DBD34', 28, 1015),
 ('1997 BMW F650 ST', 28, 1014),
 ('1957 Corvette Convertible', 27, 1013),
 ('18th century schooner', 27, 1011),
 ('1900s Vintage Tri-Plane', 28, 1009)

## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [26]:
c.execute('''select productName, count(customerNumber) numcustomer from customers
            join orders using(customerNumber)
            join orderdetails using(orderNumber)
            join products using(productCode)
            group by productName
            order by numcustomer desc;''').fetchall()

[('1992 Ferrari 360 Spider red', 53),
 ('18th Century Vintage Horse Carriage', 28),
 ('1900s Vintage Bi-Plane', 28),
 ('1900s Vintage Tri-Plane', 28),
 ('1913 Ford Model T Speedster', 28),
 ('1917 Maxwell Touring Car', 28),
 ('1926 Ford Fire Engine', 28),
 ('1928 British Royal Navy Airplane', 28),
 ('1928 Ford Phaeton Deluxe', 28),
 ('1928 Mercedes-Benz SSK', 28),
 ('1930 Buick Marquette Phaeton', 28),
 ('1932 Model A Ford J-Coupe', 28),
 ('1934 Ford V8 Coupe', 28),
 ('1936 Chrysler Airflow', 28),
 ('1936 Harley Davidson El Knucklehead', 28),
 ('1936 Mercedes-Benz 500K Special Roadster', 28),
 ('1937 Horch 930V Limousine', 28),
 ('1937 Lincoln Berline', 28),
 ('1938 Cadillac V-16 Presidential Limousine', 28),
 ('1939 Cadillac Limousine', 28),
 ('1939 Chevrolet Deluxe Coupe', 28),
 ('1940 Ford Delivery Sedan', 28),
 ('1940 Ford Pickup Truck', 28),
 ('1940s Ford truck', 28),
 ('1941 Chevrolet Special Deluxe Cabriolet', 28),
 ('1952 Alpine Renault 1300', 28),
 ('1954 Greyhound Scenicruise

In [27]:
c.execute("""select productName,
                    count(distinct customerNumber) numPurchasers
                    from products
                    join orderdetails
                    using(productCode)
                    join orders
                    using (orderNumber)
                    group by 1
                    order by numPurchasers desc
                    ;
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,productName,numPurchasers
0,1992 Ferrari 360 Spider red,40
1,1934 Ford V8 Coupe,27
2,1952 Alpine Renault 1300,27
3,1972 Alfa Romeo GTA,27
4,Boeing X-32A JSF,27


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [13]:
c.execute('''select employeeNumber, officeCode, lastName, firstName, city from employees 
            join offices using(officeCode)
            where employeeNumber in 
            (select salesRepEmployeeNumber from customers
            join orders using(customerNumber)
            join orderdetails using(orderNumber)
            join products using(productCode)
            group by productName
            having count(customerNumber) <25);''').fetchall()

[('1612', '6', 'Marsh', 'Peter', 'Sydney')]

In [28]:
c.execute("""select distinct employeeNumber,
                    officeCode,
                    o.city,
                    firstName,
                    lastName
                    from employees e
                    join offices o
                    using(officeCode)
                    join customers c
                    on e.employeeNumber = c.salesRepEmployeeNumber
                    join orders
                    using (customerNumber)
                    join orderdetails
                    using (orderNumber)
                    where productCode in (select productCode
                                                 from products
                                                 join orderdetails
                                                 using (productCode)
                                                 join orders
                                                 using (orderNumber)
                                                 group by 1
                                                 having count(distinct customerNumber) <= 20
                                         );
           """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,employeeNumber,officeCode,city,firstName,lastName
0,1370,4,Paris,Gerard,Hernandez
1,1401,4,Paris,Pamela,Castillo
2,1621,5,Tokyo,Mami,Nishi
3,1501,7,London,Larry,Bott
4,1337,4,Paris,Loui,Bondur
5,1286,3,NYC,Foon Yue,Tseng
6,1166,1,San Francisco,Leslie,Thompson
7,1165,1,San Francisco,Leslie,Jennings
8,1612,6,Sydney,Peter,Marsh
9,1188,2,Boston,Julie,Firrelli


## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [20]:
c.execute('''select employeeNumber, lastName, firstName, count(customerNumber), avg(creditLimit) from employees e 
            join customers c on e.employeeNumber= c.salesRepEmployeeNumber
            group by employeeNumber 
            having avg(creditLimit) >15000 ;''').fetchall()

[('1165', 'Jennings', 'Leslie', 6, 100433.33333333333),
 ('1166', 'Thompson', 'Leslie', 6, 65266.666666666664),
 ('1188', 'Firrelli', 'Julie', 6, 73916.66666666667),
 ('1216', 'Patterson', 'Steve', 6, 81533.33333333333),
 ('1286', 'Tseng', 'Foon Yue', 7, 66614.28571428571),
 ('1323', 'Vanauf', 'George', 8, 80887.5),
 ('1337', 'Bondur', 'Loui', 6, 86233.33333333333),
 ('1370', 'Hernandez', 'Gerard', 7, 91785.71428571429),
 ('1401', 'Castillo', 'Pamela', 10, 81340.0),
 ('1501', 'Bott', 'Larry', 8, 91187.5),
 ('1504', 'Jones', 'Barry', 9, 85511.11111111111),
 ('1611', 'Fixter', 'Andy', 5, 86060.0),
 ('1612', 'Marsh', 'Peter', 5, 92080.0),
 ('1621', 'Nishi', 'Mami', 5, 83900.0),
 ('1702', 'Gerard', 'Martin', 6, 71916.66666666667)]

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!